# Tugas Besar Machine Learning Milestone 2

## Load dataset iris

In [15]:
import pandas as pd
from sklearn import datasets,model_selection,metrics
iris = datasets.load_iris()
iris = pd.DataFrame(iris.data, columns=iris.feature_names)
iris.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [30]:
X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train = X
y_train = y
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


## Fungsi error

In [ ]:
def Errorfunc(m,b):
    global point
    n = float(len(point))
    err = 0
    for p in point:
        err += (p[0]-(m*p[1]+b))**2
    return err/n

## Fungsi-fungsi aktivasi

In [ ]:
def linear(c, Z):
    A = np.multiply(c, Z)
    return A

def sigmoid(Z):
    arr = []
    for x in Z:
        arr.append(1 / (1 + np.exp(-x)))
    return arr

def relu(Z):
    A = np.maximum(0, Z)
    return A

def softmax(Z):
    e = np.exp(Z)
    return e/e.sum()

## sample text